In [12]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

import matplotlib.pyplot as plt
import numpy as np
import kplr
from astropy.io import ascii

In [23]:
kics = ascii.read('data/kics.csv', format='no_header')['col1']

In [ ]:
import h5py
import kplr
from astropy.utils.console import ProgressBar

client = kplr.API()

# Hack for interactive use with a notebook:
try: 
    f.close()
except (NameError, ValueError):
    pass

# Create archive
f = h5py.File('data/light_curves.hdf5', 'w')

with ProgressBar(len(kics), ipython_widget=True) as bar:
    for kic in kics:
        # Find a KIC
        star = client.star(kic)

        # Download the lightcurves for this KOI.
        lightcurves = star.get_light_curves(short_cadence=False)

        # Loop over the datasets and read in the data.
        time, flux, ferr, quality = [], [], [], []
        n_columns = 4
        for lc in lightcurves:
            with lc.open() as lc_file:
                # The lightcurve data are in the first FITS HDU.
                hdu_data = lc_file[1].data

                time.append(hdu_data["time"])
                flux.append(hdu_data["sap_flux"])
                ferr.append(hdu_data["sap_flux_err"])
                quality.append(hdu_data["sap_quality"])

        data = np.vstack(list(map(np.concatenate, [time, flux, ferr, quality]))).T
        f.create_dataset(str(kic), data=data)
        f.flush()
        bar.update()